In [4]:
import pandas as pd
import numpy as np
import datetime
from geopy.distance import geodesic

In [5]:
train_data = pd.read_csv('train_data.csv')

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 76 columns):
Unnamed: 0    500000 non-null int64
sid           500000 non-null int64
pid           336021 non-null float64
req_time      500000 non-null object
o             500000 non-null object
d             500000 non-null object
plan_time     491054 non-null object
plans         491054 non-null object
p0            336021 non-null float64
p1            336021 non-null float64
p2            336021 non-null float64
p3            336021 non-null float64
p4            336021 non-null float64
p5            336021 non-null float64
p6            336021 non-null float64
p7            336021 non-null float64
p8            336021 non-null float64
p9            336021 non-null float64
p10           336021 non-null float64
p11           336021 non-null float64
p12           336021 non-null float64
p13           336021 non-null float64
p14           336021 non-null float64
p15           336021 non

# Origin and Destination Distance Calculation

#### Please install geopy package when running the code below 

In [7]:
distance = []
for i in range(train_data['o'].count()):
    lng1,lat1 = map(float,train_data['o'].iloc[i].split(','))
    lng2,lat2 = map(float,train_data['d'].iloc[i].split(','))
    #print(lng1, lat1,lng2,lat2)
    distance.append(int(geodesic((lat1, lng1),(lat2,lng2)).m))
    
distance = pd.DataFrame(data = distance,columns = ['Distance'])

# Convert req_time and click_time to Datetime format

#### Please install datetime package when running the code below

####  Req_time

In [8]:
#New list saving the converted value
req_datetime = []

#Loop through data and convert all req_time:
for i in range(train_data['req_time'].count()):
    
    req_datetime.append(datetime.datetime.strptime(train_data['req_time'].iloc[i], "%Y-%m-%d %H:%M:%S"))
    
#Change list to dataframe and merge with 'sid' column for future merging with click_time which has missing value
req_datetime = pd.concat((pd.DataFrame(data = req_datetime, columns = ['req_datetime']),train_data['sid']),axis = 1)

#### Click_time 

In [9]:
#New list savong the converted value
click_datetime = []

#As missing value exist, build a sid list saving the sid whose click_time could be converted
sid = []

#Loop through all valid click_time and save the converted value and related sid into lists.
for i in range(train_data['req_time'].count()):
    if type(train_data['click_time'].iloc[i]) != np.float:
        click_datetime.append(datetime.datetime.strptime(train_data['click_time'].iloc[i], "%Y-%m-%d %H:%M:%S"))
        sid.append(train_data['sid'].iloc[i])
        
#Change list to dataframe and merge with 'sid' list
click_datetime = pd.concat((pd.DataFrame(data = click_datetime, columns = ['click_datetime']),pd.DataFrame(sid,columns = ['sid'])),axis = 1)

#### Merge req_datetime and click_datetime and calculate the time difference

In [11]:
time = pd.merge(req_datetime, click_datetime, how = 'left',on = 'sid')

time['difference'] = time['click_datetime'] - time['req_datetime']

print('The total number of time difference is:  ', time['difference'].count())
print('The average value of time difference is:  ', time['difference'].mean())
print('The max value of time difference is:  ', time['difference'].max())
print('The min value of time difference is:  ', time['difference'].min())
print('The mode value of the time difference is:  ', time['difference'].mode()[0])

The total number of time difference is:   453336
The average value of time difference is:   0 days 00:02:08.777149
The max value of time difference is:   38 days 23:57:59
The min value of time difference is:   0 days 00:00:00
The mode value of the time difference is:   0 days 00:00:03


In [26]:
#Fill the missing value with mode value
time['difference'] = time['difference'].fillna(time['difference'].mode()[0])
print('The total number of time difference is:  ', time['difference'].count())

The total number of time difference is:   500000


# Merge distance and time difference with train_data

In [28]:
train_data = pd.concat((train_data, distance), axis = 1)

In [29]:
train_data = pd.concat((train_data, time['difference']), axis = 1)

In [33]:
train_data.head()

,Unnamed: 0,sid,pid,req_time,o,d,plan_time,plans,p0,p1,...,p60,p61,p62,p63,p64,p65,click_time,click_mode,Distance,difference
0,0,3000821,NaN,2018-11-02 17:54:30,"116.29,39.97","116.32,39.96",2018-11-02 17:54:30,"[{""distance"": 5219, ""price"": 300, ""eta"": 1367,...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-02 17:54:32,9.0,2793,00:00:02
1,1,3085857,210736.0,2018-11-16 10:53:10,"116.39,39.84","116.33,39.79",2018-11-16 10:53:10,"[{""distance"": 13864, ""price"": 600, ""eta"": 3221...",0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,2018-11-16 10:53:32,1.0,7563,00:00:22
2,2,2944522,NaN,2018-10-06 10:33:58,"116.31,39.93","116.27,40.00",2018-10-06 10:33:58,"[{""distance"": 12294, ""price"": 400, ""eta"": 2472...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-10-06 10:34:23,9.0,8490,00:00:25
3,3,559931,202427.0,2018-11-23 14:54:11,"116.27,39.88","116.39,39.90",2018-11-23 14:54:11,"[{""distance"": 14853, ""price"": 1700, ""eta"": 291...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2018-11-23 14:54:18,1.0,10501,00:00:07
4,4,2819352,172251.0,2018-10-30 11:48:41,"116.34,39.96","116.37,39.86",2018-10-30 11:48:41,"[{""distance"": 12882, ""price"": 600, ""eta"": 3211...",0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,2018-10-30 11:49:04,7.0,11395,00:00:23


####  Drop useless columns

In [36]:
train_data.drop(['Unnamed: 0', 'plan_time','click_time','o','d'], inplace = True, axis = 1)

# Note: Still missing values in pid, plans, p0-p65, click_mode

In [41]:
Missing_ratio = train_data.isnull().sum()/train_data.count() *100
Missing_ratio = Missing_ratio.sort_values(ascending = False)
Missing_ratio

p32           48.800224
p23           48.800224
p17           48.800224
p18           48.800224
p19           48.800224
p20           48.800224
p21           48.800224
p22           48.800224
p24           48.800224
pid           48.800224
p25           48.800224
p26           48.800224
p27           48.800224
p28           48.800224
p29           48.800224
p30           48.800224
p16           48.800224
p15           48.800224
p14           48.800224
p13           48.800224
p12           48.800224
p11           48.800224
p10           48.800224
p9            48.800224
p8            48.800224
p7            48.800224
p6            48.800224
p5            48.800224
p4            48.800224
p3            48.800224
                ...    
p58           48.800224
p57           48.800224
p56           48.800224
p55           48.800224
p54           48.800224
p53           48.800224
p52           48.800224
p51           48.800224
p49           48.800224
p48           48.800224
p47           48

# Ideas in extracting data from 'plans' column 

In [53]:
plans = []
sid = []
for i in range(train_data['plans'].count()):
    if type(train_data['plans'].iloc[i]) == str:
        plans.append(eval(train_data['plans'].iloc[i]))
        sid.append(train_data['sid'].iloc[i])

482265